# 設定

In [73]:
import time
import hmac
import hashlib
import base64
import requests
import json
import logging

class CoincatchClient:
    def __init__(self, api_key: str, secret_key: str, passphrase: str):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase
        self.base_url = "https://api.coincatch.com"
        self.logger = logging.getLogger(__name__)
        self._try_base64 = False  # None: 未測試, True/False: 是否需 base64 decode
    
    def _get_timestamp(self):
        return str(int(time.time() * 1000))

    def _sign(self, timestamp, method, request_path, body=""):
        """
        自動判斷 secret_key 是否需要 base64 decode。
        """
        body_str = body if body else ""
        message = f"{timestamp}{method.upper()}{request_path}{body_str}"

        # 第一次使用時嘗試兩種方式
        if self._try_base64 is None:
            # 嘗試 decode
            try:
                secret_bytes = base64.b64decode(self.secret_key)
                test_sign = base64.b64encode(
                    hmac.new(secret_bytes, message.encode('utf-8'), hashlib.sha256).digest()
                ).decode()
                self._try_base64 = True
                return test_sign
            except Exception:
                # decode 失敗，直接用原始字串
                secret_bytes = self.secret_key.encode('utf-8')
                test_sign = base64.b64encode(
                    hmac.new(secret_bytes, message.encode('utf-8'), hashlib.sha256).digest()
                ).decode()
                self._try_base64 = False
                return test_sign
        else:
            # 已經測試過，直接使用正確方式
            if self._try_base64:
                secret_bytes = base64.b64decode(self.secret_key)
            else:
                secret_bytes = self.secret_key.encode('utf-8')
            sign = base64.b64encode(
                hmac.new(secret_bytes, message.encode('utf-8'), hashlib.sha256).digest()
            ).decode()
            return sign

    def _get_headers(self, method, request_path, body=""):
        timestamp = self._get_timestamp()
        sign = self._sign(timestamp, method, request_path, body)
        return {
            "ACCESS-KEY": self.api_key,
            "ACCESS-SIGN": sign,
            "ACCESS-TIMESTAMP": timestamp,
            "ACCESS-PASSPHRASE": self.passphrase,
            "locale": "en-US",
            "Content-Type": "application/json"
        }

    # 自動補上 _UMCBL
    def _normalize_symbol(self, symbol: str) -> str:
        if not symbol.endswith("_UMCBL"):
            symbol = symbol + "_UMCBL"
        return symbol

    # === 查餘額 ===
    def get_balance(self, product_type="umcbl"):
        path = f"/api/mix/v1/account/accounts?productType={product_type}"
        method = "GET"
        headers = self._get_headers(method, path)
        try:
            res = requests.get(self.base_url + path, headers=headers)
            res.raise_for_status()
            data = res.json()
            print(data)
            if data.get("code") == "00000":
                account = data["data"][0]
                return float(account["available"])
            else:
                self.logger.error(f"API error: {data}")
                return None
        except Exception as e:
            self.logger.error(f"Request failed getting balance: {e}")
            return None

    # === 查持倉 ===
    def get_open_positions(self, symbol=None, product_type="umcbl"):
        path = f"/api/mix/v1/position/allPosition?productType={product_type}"
        if symbol:
            symbol = self._normalize_symbol(symbol)
            path += f"&symbol={symbol}"
        method = "GET"
        headers = self._get_headers(method, path)
        try:
            res = requests.get(self.base_url + path, headers=headers)
            res.raise_for_status()
            data = res.json()
            print(data)
            if data.get("code") == "00000":
                return data.get("data", [])
            else:
                self.logger.error(f"API error getting positions: {data}")
                return None
        except Exception as e:
            self.logger.error(f"Failed to get open positions: {e}")
            return None

    # === 下單 ===
    def place_order(self, symbol: str, side: str, size: float, price: float = None,
                    margin_coin="USDT", order_type="limit", time_in_force="normal",
                    take_profit=None, stop_loss=None, reduce_only=False):
        """Place an order with correct CoinCatch API spec"""
        symbol = self._normalize_symbol(symbol)
        path = "/api/mix/v1/order/placeOrder"
        method = "POST"

        # === 自動生成唯一 clientOid ===
        client_oid = f"CoinCatch#{int(time.time() * 1000)}#{uuid.uuid4().hex[:6]}"

        # === 準備請求體 ===
        body_data = {
            "symbol": str(symbol),
            "marginCoin": str(margin_coin),
            "side": str(side),                # open_long / open_short / close_long / close_short
            "orderType": str(order_type),     # market / limit
            "size": str(size),
            "clientOid": client_oid,
        }

        # market 單不需要價格，limit 則需要
        if order_type == "limit" and price is not None:
            # 自動修正精度 (ETHUSDT_UMCBL: 0.01)
            body_data["price"] = f"{float(price):.2f}"

        # timeInForceValue 是選填參數
        if time_in_force:
            body_data["timeInForceValue"] = time_in_force

        # 進階參數
        if take_profit:
            body_data["presetTakeProfitPrice"] = str(take_profit)
        if stop_loss:
            body_data["presetStopLossPrice"] = str(stop_loss)
        if reduce_only:
            body_data["reduceOnly"] = True

        body = json.dumps(body_data)
        headers = self._get_headers(method, path, body)

        try:
            response = requests.post(self.base_url + path, headers=headers, data=body)
            response.raise_for_status()
            data = response.json()
            print(data)

            if data.get("code") == "00000":
                self.logger.info(f"✅ Order placed successfully: {data}")
                return data
            else:
                self.logger.error(f"⚠️ Order failed: {data}")
                return data
        except requests.exceptions.RequestException as e:
            self.logger.error(f"Request failed placing order: {e}")
            return None

    # === 平倉 ===
    def close_position(self, symbol: str, side: str, product_type="umcbl"):
        symbol = self._normalize_symbol(symbol)
        positions = self.get_open_positions(symbol, product_type)

        if not positions:
            self.logger.info(f"No open positions found for {symbol}.")
            return [{"symbol": symbol, "status": "no_position", "message": "No open positions found."}]

        results = []
        for pos in positions:
            pos_side = pos.get("holdSide")
            pos_size = pos.get("total", "0")
            if float(pos_size) == 0:
                continue

            # 根據目前持倉方向決定正確平倉方向
            if pos_side == "long":
                close_side = "close_long"
            elif pos_side == "short":
                close_side = "close_short"
            else:
                continue

            path = "/api/mix/v1/order/placeOrder"
            method = "POST"
            body_data = {
                "symbol": symbol,
                "marginCoin": "USDT",
                "side": close_side,
                "orderType": "market",
                "size": str(pos_size)
            }

            body = json.dumps(body_data)
            headers = self._get_headers(method, path, body)

            try:
                res = requests.post(self.base_url + path, headers=headers, data=body)
                res.raise_for_status()
                data = res.json()
                print(data)

                if data.get("code") == "00000":
                    results.append({"symbol": symbol, "status": "success", "data": data})
                else:
                    results.append({"symbol": symbol, "status": "failed", "data": data})
            except Exception as e:
                results.append({"symbol": symbol, "status": "failed", "message": str(e)})

        return results

In [74]:
import os
from dotenv import load_dotenv
load_dotenv()

client = CoincatchClient(
    api_key=os.getenv("COINCATCH_API_KEY"),
    secret_key=os.getenv("COINCATCH_SECRET_KEY"),
    passphrase=os.getenv("COINCATCH_API_PASSPHRASE")
)

balance = client.get_balance()
print("USDT 餘額:", balance)

positions = client.get_open_positions("XRPUSDT")
print("持倉:", positions)

order = client.place_order("XRPUSDT", "open_long", size='1')
print("下單:", order)

close = client.close_position("XRPUSDT", "long")
print("關倉:", close)

{'code': '00000', 'msg': 'success', 'requestTime': 1762836449665, 'data': [{'marginCoin': 'USDT', 'locked': '0', 'available': '10.38738184', 'crossMaxAvailable': '10.38738184', 'fixedMaxAvailable': '10.38738184', 'maxTransferOut': '10.38738184', 'equity': '10.38738184', 'usdtEquity': '10.387381848', 'btcEquity': '0.00009819228', 'crossRiskRate': '0', 'unrealizedPL': '0', 'bonus': '0', 'crossedUnrealizedPL': None, 'isolatedUnrealizedPL': None}]}
USDT 餘額: 10.38738184
{'code': '00000', 'msg': 'success', 'requestTime': 1762836449773, 'data': []}
持倉: []


Request failed placing order: 400 Client Error: Bad Request for url: https://api.coincatch.com/api/mix/v1/order/placeOrder


下單: None
{'code': '00000', 'msg': 'success', 'requestTime': 1762836450025, 'data': []}
關倉: [{'symbol': 'XRPUSDT_UMCBL', 'status': 'no_position', 'message': 'No open positions found.'}]
